# Set up tables

In [1]:
# !pip install ipython-sql
# !pip install mysqlclient

  Using cached ipython_sql-0.4.0-py3-none-any.whl (19 kB)
Processing c:\users\tbui1\appdata\local\pip\cache\wheels\8c\76\0b\eb9eb3da7e2335e3577e3f96a0ae9f74f206e26457bd1a2bc8\prettytable-0.7.2-py3-none-any.whl
  Using cached sqlparse-0.4.1-py3-none-any.whl (42 kB)
  Using cached mysqlclient-2.0.3-cp37-cp37m-win_amd64.whl (178 kB)


In [2]:
import sqlalchemy
import MySQLdb
sqlalchemy.create_engine('mysql://root@localhost/movie_rental')
%load_ext sql
%sql mysql://root@localhost/movie_rental

In [3]:
%%sql
SHOW TABLES;

 * mysql://root@localhost/movie_rental
5 rows affected.


Tables_in_movie_rental
actors
actsin
customers
movies
renting


# Perform Analysis

## Get to know the customers

### Question 1: Where are most customers located?

In [61]:
%%sql
SELECT country, COUNT(*) as customers
FROM customers
GROUP BY country
ORDER BY customers DESC
LIMIT 5;


 * mysql://root@localhost/movie_rental
5 rows affected.


country,customers
Italy,27
Spain,16
France,13
Poland,13
Hungary,11


Understanding customers' locations helps identify the subtitles to add to the movies and where movies should be sourced. The majority of customers are in Europe (Italy, Spain, France, Poland, and Hungary in particular), which means we can add Italian, Spanish, French, etc. subtitles to the movies and source more movies from these countries.

### Question 2: What is the activation rate?

In [63]:
%%sql
SELECT COUNT(DISTINCT(r.customer_id))/COUNT(DISTINCT(c.customer_id)) as activation_rate
FROM customers c
LEFT JOIN renting r ON c.customer_id = r.customer_id;


 * mysql://root@localhost/movie_rental
1 rows affected.


activation_rate
0.9431


The activation rate is the percentage of customers who rent at least one movie amongst all registered customers. The current activation rate is 94%, which is high but there are still opportunities for improvement. The company may increase the rate by sending not-yet-activated customers marketing emails or giving them incentives to watch a movie.

### Question 3: What is the average activation time?

In [6]:
%%sql
SELECT ROUND(AVG(activation_time_days),0) as activation_time_days_avg
FROM
    #activation time of each customer = first day the customer watched a movie - account start date
    (SELECT DATEDIFF(MIN(r.date_renting), c.date_account_start) as activation_time_days, c.customer_id
    FROM renting r
    JOIN customers c ON r.customer_id = c.customer_id
    GROUP BY c.customer_id) a;

 * mysql://root@localhost/movie_rental
1 rows affected.


activation_time_days_avg
73


The current average activation time is 73 days, which means it takes an average customer more than 3 months to rent the first movie on the platform since sign-up. The company should shorten the activation time to increase engagement and reduce drop-offs. For example, customers should receive onboarding emails with movie suggestions based on their interests during the first week of sign-up.

### Question 4: What is the repeat customer rate?

In [65]:
%%sql
SELECT COUNT(customer_id)/(SELECT COUNT(DISTINCT(customer_id)) FROM customers) as repeat_cust_rate
FROM
    #number of movies each customer watched
    (SELECT customer_id, COUNT(movie_id) as movies_watched
    FROM renting
    GROUP BY customer_id) a
WHERE movies_watched >= 2;


 * mysql://root@localhost/movie_rental
1 rows affected.


repeat_cust_rate
0.8293


The repeat customer rate reflects customer satisfaction on the website. 83% customers what 2 movies or more, which means customers are relatively satisfied with the service. The company may increase this rate by recommending more relevant and highly-rated movies that may interest customers.

## Movie Performance Analysis

### Question 5: What are the most popular movies?

In [20]:
%%sql
SELECT m.title, COUNT(r.renting_id) as rentals
FROM movies m
JOIN renting r ON r.movie_id = m.movie_id
GROUP BY title
ORDER BY rentals DESC
LIMIT 10;

 * mysql://root@localhost/movie_rental
10 rows affected.


title,rentals
The Kingdom,15
Training Day,14
Harry Potter and the Half-Blood Prince,13
Monster,13
World Trade Center,13
Bridget Jones - The Edge of Reason,13
Harry Potter and the Deathly Hallows – Part 2,12
Fair Game,12
Two for the Money,11
The Two Towers,11


The company can use the list of most watched movies above to feature in the "Most Popular" recommendation tab on the website.

### Question 6: What are the highest-rated movies?

In [22]:
%%sql
SELECT m.title, AVG(r.rating) as rating
FROM movies m
JOIN renting r ON r.movie_id = m.movie_id
GROUP BY title
ORDER BY rating DESC
LIMIT 10;

 * mysql://root@localhost/movie_rental
10 rows affected.


title,rating
Ghost Rider: Spirit of Vengeance,9.0000
Morning Glory,8.4286
The Other Woman,7.5000
Harry Potter and the Philosopher's Stone,7.3333
Harry Potter and the Half-Blood Prince,7.3077
Winter's Bone,7.1111
Imagining Argentina,6.5000
Secondhand Lions,6.4286
Harry Potter and the Chamber of Secrets,6.4000
Harry Potter and the Prisoner of Azkaban,6.4000


The company can use the list of most watched movies above to feature in the "Top-Rated" recommendation tab on the website. This table also shows that most highest-rated movies only receive 6-7/10 rating, which means the company should encourge more customer reviews and give incentives to increase movies' ratings.

### Question 7: Which genre is popular and receives high ratings?

In [27]:
%%sql
SELECT m.genre, AVG(r.rating) as rating, COUNT(DISTINCT(r.movie_id)) as movies, COUNT(r.renting_id) as rentals
FROM movies m
JOIN renting r ON r.movie_id = m.movie_id
GROUP BY genre
ORDER BY rating DESC;

 * mysql://root@localhost/movie_rental
8 rows affected.


genre,rating,movies,rentals
Art House & International,6.5000,1,8
Action & Adventure,6.2500,3,16
Science Fiction & Fantasy,5.2421,11,95
Animation,4.6667,2,12
Drama,4.5580,36,319
Other,4.4000,2,20
Comedy,3.3768,10,69
Mystery & Suspense,3.1538,6,39


Drama is the most popular genre but receives a low rating. The company may want to source more popular and highly-rated movies in this genre. In contrast, more movies in the Art House & International and Action & Adventure genre should be added because they receive the highest ratings.

### Question 8: Would customers prefer to watch longer or shorter movies?

In [47]:
%%sql
SELECT AVG(runtime)
FROM movies;

 * mysql://root@localhost/movie_rental
1 rows affected.


AVG(runtime)
121.0141


In [45]:
%%sql
SELECT COUNT(a.title) as longer_movies, SUM(a.rentals) as rentals, AVG(a.rating) as rating
FROM
    #movies with longer runtime
    (SELECT m.title, COUNT(r.renting_id) as rentals, AVG(r.rating) as rating
    FROM renting r
    JOIN movies m ON r.movie_id = m.movie_id
    WHERE m.runtime >= (SELECT AVG(runtime) FROM movies)
    GROUP BY title) a;

 * mysql://root@localhost/movie_rental
1 rows affected.


longer_movies,rentals,rating
31,269,4.36059677


In [66]:
%%sql
SELECT COUNT(a.title) as shorter_movies, SUM(a.rentals) as rentals, AVG(a.rating) as rating
FROM
    #movies with shorter runtime
    (SELECT m.title, COUNT(r.renting_id) as rentals, AVG(r.rating) as rating
    FROM renting r
    JOIN movies m ON r.movie_id = m.movie_id
    WHERE m.runtime < (SELECT AVG(runtime) FROM movies)
    GROUP BY title) a;

 * mysql://root@localhost/movie_rental
1 rows affected.


shorter_movies,rentals,rating
40,309,4.55002250


The average movie runtime is 121 minutes. Let's assume that longer movies are equal or above the average and shorter movies are below. 
Longer movies have slightly higher traction than shorter movies (9 vs. 8 rental/movie). However, the average rating of shorter movies is higher than the counterpart. This means the runtime may not play an important role in the company's movie selection.

### Question 9: Are highest rated movies properly priced?

In [48]:
%%sql
SELECT AVG(renting_price)
FROM movies;

 * mysql://root@localhost/movie_rental
1 rows affected.


AVG(renting_price)
2.2958


In [59]:
%%sql
SELECT m.title, AVG(r.rating) as rating, m.renting_price
FROM movies m
JOIN renting r ON r.movie_id = m.movie_id
WHERE m.renting_price < (SELECT AVG(renting_price) FROM movies)
GROUP BY title
ORDER BY rating DESC
LIMIT 20;

 * mysql://root@localhost/movie_rental
20 rows affected.


title,rating,renting_price
Ghost Rider: Spirit of Vengeance,9.0000,2
The Other Woman,7.5000,2
Harry Potter and the Half-Blood Prince,7.3077,2
Secondhand Lions,6.4286,2
Harry Potter and the Chamber of Secrets,6.4000,2
Harry Potter and the Prisoner of Azkaban,6.4000,2
Failure to Launch,6.0000,2
Young Adult,5.8889,2
The Hunger Games,5.6667,2
Rio 2,5.3333,2


In order to increase revenue, the company may either increase the number of rentals or price per rental. Movies with high ratings are most likely those with a higher than average price. The average renting price is 2.2 USD/movie. The table above shows the movies with high ratings but are priced below the average. The company is recommended to increase the renting price of movies with 6.0 rating or above to $3 and monitor the performance.

### Question 10: Who are the most popular actors?

In [16]:
%%sql
SELECT a.name as actor, COUNT(DISTINCT(b.movie_id)) as movies, COUNT(r.renting_id) as rentals
FROM actors a
JOIN actsin b ON a.actor_id = b.actor_id
JOIN renting r ON b.movie_id = r.movie_id
GROUP BY actor
ORDER BY rentals DESC
LIMIT 10;

 * mysql://root@localhost/movie_rental
10 rows affected.


actor,movies,rentals
Emma Watson,8,71
Rupert Grint,8,71
Daniel Radcliffe,8,71
Jamie Foxx,4,42
Al Pacino,4,37
Helen Hunt,4,37
Charlize Theron,4,33
Nicolas Cage,4,31
Elijah Wood,4,30
Catherine Keener,3,27


Actors play a pivotal role in audience's movie picking decisions. Knowing that Emma Watson, Rupert Grint, and Daniel Radcliffe are the most popular actors, the company can add more movies featuring these actors in the movie list.